# Validação: XLM-RoBERTa

**XLM-RoBERTa Large - Multilingual**

## 📊 Histórico
- Score 0.68767 com Mean Pooling (3º melhor transformer)
- CLS pooling não testado

## 🎯 Objetivo
Testar diferentes estratégias de pooling e Focal Loss.

## 📊 Configurações Testadas
1. **Pooling:** CLS, Mean, Max, Weighted Mean
2. **Loss:** CrossEntropy vs Focal Loss
3. **Model Size:** base vs large

---

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from transformers import AutoModel, AutoTokenizer, AutoConfig
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if os.path.exists('/kaggle/input'):
    DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
    def find_model_path():
        base = '/kaggle/input'
        def search_dir(directory, depth=0, max_depth=10):
            if depth > max_depth: return None
            try:
                for item in os.listdir(directory):
                    path = os.path.join(directory, item)
                    if os.path.isdir(path) and os.path.exists(os.path.join(path, 'config.json')):
                        return path
                    result = search_dir(path, depth + 1, max_depth) if os.path.isdir(path) else None
                    if result: return result
            except: pass
            return None
        return search_dir(base)
    MODEL_PATH = find_model_path()
else:
    DATA_DIR = '../data'
    MODEL_PATH = 'xlm-roberta-large'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
print(f'Model: {MODEL_PATH}')

In [ ]:
# ===== DADOS =====
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['report'].tolist(),
    train_df['target'].tolist(),
    test_size=0.2,
    stratify=train_df['target'],
    random_state=SEED
)

print(f'Train: {len(train_texts)}, Val: {len(val_texts)}')

In [ ]:
# ===== MODELO COM DIFERENTES POOLINGS =====
class XLMRoBERTaWithPooling(nn.Module):
    def __init__(self, model_path, num_labels=7, pooling='cls'):
        super().__init__()
        self.pooling = pooling
        self.encoder = AutoModel.from_pretrained(model_path, local_files_only=True)
        hidden_size = self.encoder.config.hidden_size
        self.classifier = nn.Linear(hidden_size, num_labels)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        hidden = outputs.last_hidden_state
        
        if self.pooling == 'cls':
            pooled = hidden[:, 0, :]
        elif self.pooling == 'mean':
            mask = attention_mask.unsqueeze(-1).expand(hidden.size()).float()
            pooled = (hidden * mask).sum(1) / mask.sum(1)
        elif self.pooling == 'max':
            mask = attention_mask.unsqueeze(-1).expand(hidden.size()).float()
            hidden[mask == 0] = -1e9
            pooled = hidden.max(1)[0]
        else:
            pooled = hidden[:, 0, :]
        
        pooled = self.dropout(pooled)
        logits = self.classifier(pooled)
        
        loss = None
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
        
        return type('Output', (), {'loss': loss, 'logits': logits})()

In [ ]:
# ===== FOCAL LOSS =====
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        
    def forward(self, inputs, targets):
        ce_loss = nn.functional.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()

In [ ]:
# ===== DATASET =====
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        enc = self.tokenizer(
            str(self.texts[idx]),
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
        )
        return {
            'input_ids': enc['input_ids'].squeeze(),
            'attention_mask': enc['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [ ]:
# ===== TREINO MANUAL =====
def train_model(pooling='cls', use_focal=False, epochs=3, lr=2e-5, batch_size=8):
    print(f"\n{'='*60}")
    print(f"Pooling: {pooling}, Focal: {use_focal}")
    print(f"{'='*60}")
    
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
    
    train_ds = TextDataset(train_texts, train_labels, tokenizer)
    val_ds = TextDataset(val_texts, val_labels, tokenizer)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size*2)
    
    model = XLMRoBERTaWithPooling(MODEL_PATH, num_labels=7, pooling=pooling).to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    
    if use_focal:
        criterion = FocalLoss(gamma=2.0, alpha=0.25)
    else:
        criterion = nn.CrossEntropyLoss()
    
    best_f1 = 0
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}'):
            batch = {k: v.to(device) for k, v in batch.items()}
            labels = batch.pop('labels')
            
            optimizer.zero_grad()
            outputs = model(**batch)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        # Validação
        model.eval()
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for batch in val_loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                labels = batch.pop('labels')
                outputs = model(**batch)
                preds = outputs.logits.argmax(dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        f1 = f1_score(all_labels, all_preds, average='macro')
        print(f'Epoch {epoch+1}: Loss={total_loss/len(train_loader):.4f}, F1={f1:.5f}')
        
        if f1 > best_f1:
            best_f1 = f1
    
    print(f'\nMelhor F1: {best_f1:.5f}')
    print(classification_report(all_labels, all_preds))
    
    del model
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    return best_f1, all_preds

In [ ]:
# ===== EXPERIMENTOS DE POOLING =====
results = {}

# CLS Pooling
f1_cls, _ = train_model(pooling='cls', use_focal=False)
results['CLS'] = f1_cls

In [ ]:
# Mean Pooling
f1_mean, _ = train_model(pooling='mean', use_focal=False)
results['Mean'] = f1_mean

In [ ]:
# Max Pooling
f1_max, _ = train_model(pooling='max', use_focal=False)
results['Max'] = f1_max

In [ ]:
# Mean + Focal Loss
f1_mean_focal, _ = train_model(pooling='mean', use_focal=True)
results['Mean+Focal'] = f1_mean_focal

In [ ]:
# ===== RESUMO =====
print("\n" + "="*60)
print("📊 RESUMO - XLM-RoBERTa Validation")
print("="*60)

for name, f1 in sorted(results.items(), key=lambda x: -x[1]):
    print(f"{name:<20} {f1:.5f}")

print(f"\n📝 Score anterior (Mean): 0.68767")
print(f"📝 Referência (BERTimbau v4): 0.82073")